# Stage 1: Define Information Need and Evaluate Data

# Stage 2: Building minimum pipeline

### Approche: 

According to the results of stage 1, we first extracted brand, color, and material information from the product attributes file, and merged them as well as the product description into training data file. 

Later on we splited text data in the training data into arrays of words under each column, and used Word2Vec as the first transformer to distribute each data set into different and continuous dimensions. 

Next, we create three features dirived from cosine similarities between Search Term and Product Title, Product Description or Product Attributes respectively. 

At last, we assembled the three features and put them into Random Forest Regression estimator to build our first experimental pipeline which is later used on testing data, to predict the relevacy score of given search term and product in the testing data file.


In [24]:
import pandas as pd
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import udf
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, DoubleType
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import HashingTF, Tokenizer
import pandas as pd
from pyspark.sql import SQLContext
from pyspark.sql import Row, functions
from pyspark.sql.functions import udf, col
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
from pyspark.ml.feature import Word2Vec
from pyspark.sql import Row, functions
from pyspark.sql.functions import udf, col
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.sql.types import DoubleType
import math
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import RandomForestRegressor



In [25]:
# mergefunction to use only brand, color and material in product_attributes
def mergeFunction(attr):
    names = attributes["name"]
    values = attributes["value"]
    result = []
    for name, value in zip(names, values):
        if "Brand".lower() or "Color".lower() or "Material".lower() in name.lower():
            result.append(value)
    return " ".join(result)


In [26]:
# Function to find cosine similarity between vectors
def cosineSimilarity(v1,v2):
    sumOfXs = 0
    sumOfYs = 0
    sumOfXYs = 0
    for i in range(len(v1)):
        sumOfXs += v1[i] * v1[i]
        sumOfYs +=  v2[i] * v2[i]
        sumOfXYs += v2[i] * v1[i]
        
    return float(sumOfXYs / math.sqrt(sumOfXs * sumOfYs))

idfUDF=udf(cosineSimilarity, DoubleType())

In [37]:
# Import all files
test_data = pd.read_csv("/home/jiawenz1_c4gcp/test.csv", encoding = 'ISO-8859-1').head(100)
train_data = pd.read_csv('/home/jiawenz1_c4gcp/train.csv', encoding = 'ISO-8859-1').head(100)
product_description = pd.read_csv('/home/jiawenz1_c4gcp/product_descriptions.csv', encoding = 'ISO-8859-1').head(100)
attributes = pd.read_csv('/home/jiawenz1_c4gcp/attributes.csv', encoding = 'ISO-8859-1').head(100)

###  Processing training data

In [38]:

# merge train dataframe and description dataframe
train_and_description = pd.merge(train_data, product_description, how="left", on="product_uid")
attributes.dropna(how="all", inplace=True)
attributes["product_uid"] = attributes["product_uid"].astype(int)
attributes["value"] = attributes["value"].astype(str)
product_attributes = attributes.groupby("product_uid").apply(mergeFunction)
product_attributes = product_attributes.reset_index(name="product_attributes")


In [39]:

# merge train_description dataframe and attribute dataframe
train_description_attributes = pd.merge(train_and_description, product_attributes, how="left", on="product_uid")
fields = [StructField("id", StringType(), True), StructField("product_uid", StringType(), True), StructField("product_title", StringType(), True), StructField("search_term", StringType(), True)
         , StructField("relevance", StringType(), True), StructField("product_description", StringType(), True), StructField("product_attributes", StringType(), True)]
mergeSchema = StructType(fields)
# convert pandas dataframe to spark dataframe
training = sqlContext.createDataFrame(train_description_attributes, mergeSchema)
training.show(5)


+---+-----------+--------------------+------------------+---------+--------------------+--------------------+
| id|product_uid|       product_title|       search_term|relevance| product_description|  product_attributes|
+---+-----------+--------------------+------------------+---------+--------------------+--------------------+
|  2|     100001|Simpson Strong-Ti...|     angle bracket|      3.0|Not only do angle...|Versatile connect...|
|  3|     100001|Simpson Strong-Ti...|         l bracket|      2.5|Not only do angle...|Versatile connect...|
|  9|     100002|BEHR Premium Text...|         deck over|      3.0|BEHR Premium Text...|Versatile connect...|
| 16|     100005|Delta Vero 1-Hand...|  rain shower head|     2.33|Update your bathr...|                 NaN|
| 17|     100005|Delta Vero 1-Hand...|shower only faucet|     2.67|Update your bathr...|                 NaN|
+---+-----------+--------------------+------------------+---------+--------------------+--------------------+
only showi

In [40]:
 
# # Processing Dataframe
split_col = functions.split(training[3], " ")
training = training.withColumn('search_term', split_col)
split_col = functions.split(training[2], " ")
training = training.withColumn('product_title', split_col)
split_col = functions.split(training[5], " ")
training = training.withColumn('product_description', split_col)
split_col = functions.split(training[6], " ")
training = training.withColumn('product_attributes', split_col)
to_double = training[4].cast(DoubleType())
training = training.withColumn('relevance', to_double)
training.show(5)


+---+-----------+--------------------+--------------------+---------+--------------------+--------------------+
| id|product_uid|       product_title|         search_term|relevance| product_description|  product_attributes|
+---+-----------+--------------------+--------------------+---------+--------------------+--------------------+
|  2|     100001|[Simpson, Strong-...|    [angle, bracket]|      3.0|[Not, only, do, a...|[Versatile, conne...|
|  3|     100001|[Simpson, Strong-...|        [l, bracket]|      2.5|[Not, only, do, a...|[Versatile, conne...|
|  9|     100002|[BEHR, Premium, T...|        [deck, over]|      3.0|[BEHR, Premium, T...|[Versatile, conne...|
| 16|     100005|[Delta, Vero, 1-H...|[rain, shower, head]|     2.33|[Update, your, ba...|               [NaN]|
| 17|     100005|[Delta, Vero, 1-H...|[shower, only, fa...|     2.67|[Update, your, ba...|               [NaN]|
+---+-----------+--------------------+--------------------+---------+--------------------+--------------

In [41]:
# Use word2Vec to do feature extraction
word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="search_term", outputCol="search_term_array_features")
model = word2Vec.fit(training)
training = model.transform(training)
word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="product_title", outputCol="product_title_array_features")
model = word2Vec.fit(training)
training = model.transform(training)
word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="product_description", outputCol="product_description_array_features")
model = word2Vec.fit(training)
training = model.transform(training)
word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="product_attributes", outputCol="product_attributes_array_features")
model = word2Vec.fit(training)
training = model.transform(training)
training.show(5)


+---+-----------+--------------------+--------------------+---------+--------------------+--------------------+--------------------------+----------------------------+----------------------------------+---------------------------------+
| id|product_uid|       product_title|         search_term|relevance| product_description|  product_attributes|search_term_array_features|product_title_array_features|product_description_array_features|product_attributes_array_features|
+---+-----------+--------------------+--------------------+---------+--------------------+--------------------+--------------------------+----------------------------+----------------------------------+---------------------------------+
|  2|     100001|[Simpson, Strong-...|    [angle, bracket]|      3.0|[Not, only, do, a...|[Versatile, conne...|      [-0.0027407575398...|        [-0.0210386053659...|              [-0.0264213297573...|             [-0.0068195634406...|
|  3|     100001|[Simpson, Strong-...|        [l, br

In [42]:

# call the idfUDF function to calculate the Cosine Similarity between each two vectors
training = training.withColumn("searchAndTitle", idfUDF("search_term_array_features","product_title_array_features"))
training = training.withColumn("searchAndDescription", idfUDF("search_term_array_features","product_description_array_features"))
training = training.withColumn("searchAndAttributes", idfUDF("search_term_array_features","product_attributes_array_features"))


# combine three features into a dictionary
features=["searchAndTitle", "searchAndDescription", "searchAndAttributes"]
assembler_features = VectorAssembler(inputCols=features, outputCol='features')
training = assembler_features.transform(training)
training.show(5)
training.select("features").show(10)


+---+-----------+--------------------+--------------------+---------+--------------------+--------------------+--------------------------+----------------------------+----------------------------------+---------------------------------+--------------------+--------------------+--------------------+--------------------+
| id|product_uid|       product_title|         search_term|relevance| product_description|  product_attributes|search_term_array_features|product_title_array_features|product_description_array_features|product_attributes_array_features|      searchAndTitle|searchAndDescription| searchAndAttributes|            features|
+---+-----------+--------------------+--------------------+---------+--------------------+--------------------+--------------------------+----------------------------+----------------------------------+---------------------------------+--------------------+--------------------+--------------------+--------------------+
|  2|     100001|[Simpson, Strong-...

### Processing testing data

In [43]:
# Processing the testing data, tokeninzing columns
test_and_description = pd.merge(test_data, product_description, how="left", on="product_uid")
test_description_attributes = pd.merge(test_and_description, product_attributes, how="left", on="product_uid")
fields = [StructField("id", StringType(), True), StructField("product_uid", StringType(), True), StructField("product_title", StringType(), True), StructField("search_term", StringType(), True)
         , StructField("product_description", StringType(), True), StructField("product_attributes", StringType(), True)]
mergeSchema = StructType(fields)
testing = sqlContext.createDataFrame(test_description_attributes, mergeSchema)
split_col = functions.split(testing[3], " ")
testing = testing.withColumn('search_term', split_col)
split_col = functions.split(testing[2], " ")
testing = testing.withColumn('product_title', split_col)
split_col = functions.split(testing[4], " ")
testing = testing.withColumn('product_description', split_col)
split_col = functions.split(testing[5], " ")
testing = testing.withColumn('product_attributes', split_col)
testing.show(5)


+---+-----------+--------------------+--------------------+--------------------+--------------------+
| id|product_uid|       product_title|         search_term| product_description|  product_attributes|
+---+-----------+--------------------+--------------------+--------------------+--------------------+
|  1|     100001|[Simpson, Strong-...|[90, degree, brac...|[Not, only, do, a...|[Versatile, conne...|
|  4|     100001|[Simpson, Strong-...|[metal, l, brackets]|[Not, only, do, a...|[Versatile, conne...|
|  5|     100001|[Simpson, Strong-...|[simpson, sku, able]|[Not, only, do, a...|[Versatile, conne...|
|  6|     100001|[Simpson, Strong-...|[simpson, strong,...|[Not, only, do, a...|[Versatile, conne...|
|  7|     100001|[Simpson, Strong-...|[simpson, strong,...|[Not, only, do, a...|[Versatile, conne...|
+---+-----------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [44]:
# Converting all tokens to vectors
word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="search_term", outputCol="search_term_array_features")
model = word2Vec.fit(testing)
testing = model.transform(testing)
word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="product_title", outputCol="product_title_array_features")
model = word2Vec.fit(testing)
testing = model.transform(testing)
word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="product_description", outputCol="product_description_array_features")
model = word2Vec.fit(testing)
testing = model.transform(testing)
word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="product_attributes", outputCol="product_attributes_array_features")
model = word2Vec.fit(testing)
testing = model.transform(testing)
testing.show(5)


+---+-----------+--------------------+--------------------+--------------------+--------------------+--------------------------+----------------------------+----------------------------------+---------------------------------+
| id|product_uid|       product_title|         search_term| product_description|  product_attributes|search_term_array_features|product_title_array_features|product_description_array_features|product_attributes_array_features|
+---+-----------+--------------------+--------------------+--------------------+--------------------+--------------------------+----------------------------+----------------------------------+---------------------------------+
|  1|     100001|[Simpson, Strong-...|[90, degree, brac...|[Not, only, do, a...|[Versatile, conne...|      [0.02333864212657...|        [3.53743555024266...|              [0.03069015102219...|             [0.04154385153267...|
|  4|     100001|[Simpson, Strong-...|[metal, l, brackets]|[Not, only, do, a...|[Versatile, 

In [45]:
# Finding Cosine Similarity between vectors
testing = testing.withColumn("searchAndTitle", idfUDF("search_term_array_features","product_title_array_features"))
testing = testing.withColumn("searchAndDescription", idfUDF("search_term_array_features","product_description_array_features"))
testing = testing.withColumn("searchAndAttributes", idfUDF("search_term_array_features","product_attributes_array_features"))
features=["searchAndTitle", "searchAndDescription", "searchAndAttributes"]
assembler_features = VectorAssembler(inputCols=features, outputCol='features')
testing = assembler_features.transform(testing)
testing.show(5)
testing.select("features").show(10)

+---+-----------+--------------------+--------------------+--------------------+--------------------+--------------------------+----------------------------+----------------------------------+---------------------------------+-------------------+--------------------+--------------------+--------------------+
| id|product_uid|       product_title|         search_term| product_description|  product_attributes|search_term_array_features|product_title_array_features|product_description_array_features|product_attributes_array_features|     searchAndTitle|searchAndDescription| searchAndAttributes|            features|
+---+-----------+--------------------+--------------------+--------------------+--------------------+--------------------------+----------------------------+----------------------------------+---------------------------------+-------------------+--------------------+--------------------+--------------------+
|  1|     100001|[Simpson, Strong-...|[90, degree, brac...|[Not, only,

### Building pipeline using assembled 3 features and RandomForestRegressor as estimator, train it with training data, and use it on testing data.

In [46]:
# Using thr Random Forest Regressor on the data
rf = RandomForestRegressor(featuresCol="features",labelCol='relevance', numTrees=20, maxDepth=5)
pipeline = Pipeline(stages=[rf])

# train the model
model = pipeline.fit(training)

# testing
prediction = model.transform(testing)
prediction.show(5)

+---+-----------+--------------------+--------------------+--------------------+--------------------+--------------------------+----------------------------+----------------------------------+---------------------------------+-------------------+--------------------+--------------------+--------------------+------------------+
| id|product_uid|       product_title|         search_term| product_description|  product_attributes|search_term_array_features|product_title_array_features|product_description_array_features|product_attributes_array_features|     searchAndTitle|searchAndDescription| searchAndAttributes|            features|        prediction|
+---+-----------+--------------------+--------------------+--------------------+--------------------+--------------------------+----------------------------+----------------------------------+---------------------------------+-------------------+--------------------+--------------------+--------------------+------------------+
|  1|     100

In [47]:
# Writing prediction to a csv file on instance
submission=prediction.select("id","prediction")
submission.show(5)
submission = submission.toPandas()

# output the result into a csv file
submission.to_csv('/home/jiawenz1_c4gcp/answer.csv', index=False)

+---+------------------+
| id|        prediction|
+---+------------------+
|  1| 2.316199810727489|
|  4|2.8700816017316018|
|  5| 2.480921273586955|
|  6| 2.565411425102106|
|  7|2.4709336473243275|
+---+------------------+
only showing top 5 rows



### Observation and Analysis: 

All predictions from this pipeline landed in the range of 2.30 - 2.46, this biased prediction might resulted form:

(1)uneven length of text under each column, for example the product description is much longer than extracted attributes or product title, the data intensity in much lower in  the product description after it is distributed into 10 dimensions than that of attribues and product title;

(2)cosine similarity can only convey the angles between objectives and features, but not the distance between features and objectives in the 10 dimension space;

(3) We didn't choose the optimum number of dimension for each variables when using the Word2Vec

# Stage 4: Improve transformers

Based on the observation in stage 2, we took 3 actions to improve our transformers:

(1) We added Euclidean distance and used it with Word2Vec so as to take distance information into the model;

(2) We added Word match function as transformer to obtain absolute values to get rid off the issue about the different length of product_description, product_title, and product_attribues, which should not be a matter in this problem.

(2) We used Word2Vec + CosineSimilarity and Word2Vec + EuclideanDistance only on product_title and product_attributs that have similar length, and used WordMatch for all three variables with search_term.

In the end we obtained 7 features and trained a new model with them.

In [ ]:
# Function to find euclidean distance between vectors
def euclideanDistance(v1,v2):
    dist = 0
    if len(v1) < len(v2):
        for i in range(len(v2)):
            if i < len(v1):
                dist += (v2[i] - v1[i]) * (v2[i] - v1[i])
            else:
                dist += v2[i] * v2[i]
    else:
        for i in range(len(v1)):
            if i < len(v2):
                dist += (v2[i] - v1[i]) * (v2[i] - v1[i])
            else:
                dist += v1[i] * v1[i]
    return float(math.sqrt(dist))

edUDF=udf(euclideanDistance, DoubleType())
# v1 = [10, 10, 10]
# v2 = [5, 5]
# print (euclideanDistance(v1, v2))

In [ ]:
# Function to find how many words match between vectors
def numberOfWordsMatched(v1,v2):
    l1 = len(v1)
    l2 = len(v2)
    match = 0
    for i in range(l1):
        v1[i] = v1[i].lower()
        for j in range(l2):
            v2[j] = v2[j].lower()
            if v1[i] == v2[j]:
                match += 2
            elif v1[i] in v2[j]:
                match += 1
            elif v2[j] in v1[i]:
                match += 1
            else:
                match += 0
    return match
matchUDF=udf(numberOfWordsMatched, IntegerType())

# v1 = ["hi", "my", "name"]
# v2 = ["hi", "abc", "names"]
# print (numberOfWordsMatched(v1,v2))

### Processing training data

In [ ]:
# call the idfUDF function to calculate the Cosine Similarity between each two vectors
training = training.withColumn("searchAndTitle", idfUDF("search_term_array_features","product_title_array_features"))
training = training.withColumn("searchAndAttributes", idfUDF("search_term_array_features","product_attributes_array_features"))

# call the edUDF function to calculate the Eucledian Distance between each two vectors
training = training.withColumn("searchAndTitleED", edUDF("search_term_array_features","product_title_array_features"))
training = training.withColumn("searchAndAttributesED", edUDF("search_term_array_features","product_attributes_array_features"))

# call the matchUDF function to calculate the Eucledian Distance between each two vectors
training = training.withColumn("searchAndTitleMD", matchUDF("search_term_","product_title_"))
training = training.withColumn("searchAndAttributesMD", matchUDF("search_term_","product_attributes_"))
training = training.withColumn("searchAndDescriptionMD", matchUDF("search_term_","product_description_"))

# combine three features into a dictionary
features=["searchAndTitleED", "searchAndTitleMD", "searchAndAttributesED", "searchAndTitle", "searchAndAttributes", "searchAndAttributesMD", "searchAndDescriptionMD"]
assembler_features = VectorAssembler(inputCols=features, outputCol='features')
training = assembler_features.transform(training)
training.select("features").show(10)

In [ ]:
+--------------------+
|            features|
+--------------------+
|[0.10091605379817...|
|[0.07750586746402...|
|[0.06845418730860...|
|[0.06421587612470...|
|[0.05030394370615...|
|[0.10812486168510...|
|[0.12025158840720...|
|[0.12464254207317...|
|[0.08713947684420...|
|[0.23118286971610...|
+--------------------+
only showing top 10 rows

### Processing testing data

In [ ]:
# Finding Cosine Similarity between vectors
testing = testing.withColumn("searchAndTitle", idfUDF("search_term_array_features","product_title_array_features"))
testing = testing.withColumn("searchAndAttributes", idfUDF("search_term_array_features","product_attributes_array_features"))

# call the edUDF function to calculate the Eucledian Distance between each two vectors
testing = testing.withColumn("searchAndTitleED", edUDF("search_term_array_features","product_title_array_features"))
testing = testing.withColumn("searchAndAttributesED", edUDF("search_term_array_features","product_attributes_array_features"))

# call the matchUDF function to calculate the Eucledian Distance between each two vectors
testing = testing.withColumn("searchAndTitleMD", matchUDF("search_term_","product_title_"))
testing = testing.withColumn("searchAndAttributesMD", matchUDF("search_term_","product_attributes_"))
testing = testing.withColumn("searchAndDescriptionMD", matchUDF("search_term_","product_description_"))

# combine seven features into a dictionary
features=["searchAndTitleED", "searchAndTitleMD", "searchAndAttributesED", "searchAndTitle", "searchAndAttributes", "searchAndAttributesMD", "searchAndDescriptionMD"]

assembler_features = VectorAssembler(inputCols=features, outputCol='features')
testing = assembler_features.transform(testing)
testing.show(5)
testing.select("features").show(10)

In [ ]:
+---+-----------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------------+----------------------------+---------------------------------+-------------------+-------------------+-------------------+---------------------+----------------+---------------------+----------------------+--------------------+
| id|product_uid|        search_term_|      product_title_|product_description__|product_description_| product_attributes_|search_term_array_features|product_title_array_features|product_attributes_array_features|     searchAndTitle|searchAndAttributes|   searchAndTitleED|searchAndAttributesED|searchAndTitleMD|searchAndAttributesMD|searchAndDescriptionMD|            features|
+---+-----------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------------+----------------------------+---------------------------------+-------------------+-------------------+-------------------+---------------------+----------------+---------------------+----------------------+--------------------+
|  1|     100001|[90, degree, brac...|[simpson, strong-...| [not, only, do, a...|[angles, make, jo...|[simpson, strong-...|      [-0.0024249718214...|        [0.00304099731147...|             [0.01872349058329...|  0.093711767633554| -0.522947684207269|0.08035624995514339|  0.13405254361682262|               0|                    0|                     2|[0.08035624995514...|
|  4|     100001|[metal, l, brackets]|[simpson, strong-...| [not, only, do, a...|[angles, make, jo...|[simpson, strong-...|      [-3.4288999934991...|        [0.00304099731147...|             [0.01872349058329...|-0.3534827826772235|-0.4786740616270032|0.09198408567391715|   0.1263812523222371|               1|                    2|                    16|[0.09198408567391...|
|  5|     100001|[simpson, sku, able]|[simpson, strong-...| [not, only, do, a...|[angles, make, jo...|[simpson, strong-...|      [0.03450785391032...|        [0.00304099731147...|             [0.01872349058329...| 0.4075075070864091| 0.5159938517326743|0.07981403728049065|  0.08806955741436671|               2|                    2|                     2|[0.07981403728049...|
|  6|     100001|[simpson, strong,...|[simpson, strong-...| [not, only, do, a...|[angles, make, jo...|[simpson, strong-...|      [0.03005663189105...|        [0.00304099731147...|             [0.01872349058329...|0.08004009342072974| 0.5270192377343106|0.07978304307903628|  0.08175556069006426|               7|                   12|                    93|[0.07978304307903...|
|  7|     100001|[simpson, strong,...|[simpson, strong-...| [not, only, do, a...|[angles, make, jo...|[simpson, strong-...|      [0.01446894637774...|        [0.00304099731147...|             [0.01872349058329...|-0.2801532572581965| 0.4502219749129761|0.09059617577360442|  0.08605488307188348|               4|                    4|                     8|[0.09059617577360...|
+---+-----------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------------+----------------------------+---------------------------------+-------------------+-------------------+-------------------+---------------------+----------------+---------------------+----------------------+--------------------+
only showing top 5 rows

+--------------------+
|            features|
+--------------------+
|[0.08035624995514...|
|[0.09198408567391...|
|[0.07981403728049...|
|[0.07978304307903...|
|[0.09059617577360...|
|[0.12605135382358...|
|[0.24375789452819...|
|[0.27385664587816...|
|[0.25270126794471...|
|[0.07805833249089...|
+--------------------+
only showing top 10 rows


### Building pipeline using assembled 7 features and RandomForestRegressor as estimator, train it with training data, and use it on testing data.

In [ ]:
# Using thr Random Forest Regressor on the data
rf = RandomForestRegressor(featuresCol="features",labelCol='relevance', numTrees=20, maxDepth=5)
pipeline = Pipeline(stages=[rf])

# train the model
model = pipeline.fit(training)

# testing
prediction = model.transform(testing)
prediction.show(5)

In [ ]:
# Writing prediction to a csv file on instance
submission=prediction.select("id","prediction")
submission.show(100)
submission = submission.toPandas()

# output the result into a csv file
submission.to_csv('/home/jiawenz1_c4gcp/answer.csv', index=False)

In [ ]:
+---+------------------+
| id|        prediction|
+---+------------------+
|  1|2.2531105570770817|
|  4| 2.516205260017714|
|  5| 2.412751772787061|
|  6|2.1677275793650796|
|  7| 2.564859307760412|
|  8| 2.212191500385579|
| 10|2.6471629641729106|
| 11| 2.492132776452756|
| 12| 2.482303198701883|
| 13|2.6193777209717926|
| 14| 2.711583790979911|
| 15|  2.57199076228244|
| 19|1.9151565923207223|
| 22|2.6101884256926033|
| 24|2.4518054676506185|
| 25| 2.436263331926154|
| 26| 2.113331976576055|
| 28|2.4895287863373605|
| 29| 2.206066819693464|
| 30|2.3932106557464983|
| 31|2.5763032222239843|
| 32|2.4656891664266207|
| 33| 2.466463631936975|
| 36| 2.424626153453097|
| 39| 2.250550207324413|
| 40|1.7955178571428572|
| 41| 2.473871612864385|
| 42|2.4394108173186435|
| 43| 2.371889957996507|
| 44|2.5031172417230647|
| 45|2.7344864137339733|
| 46|2.6756648451065232|
| 47| 1.764080357142857|
| 49|2.6200956479104724|
| 50|2.5818523734218033|
| 52| 2.556875901291691|
| 53| 2.405162823234474|
| 54| 2.159102978033318|
| 55| 2.606694042533913|
| 56| 2.436363143747295|
| 57|2.4425536634053087|
| 58|  2.48120410904154|
| 59|2.4781460623997367|
| 60|2.0468611119144016|
| 61|2.5714128027112695|
| 62|2.5315011553963203|
| 63| 2.605281762527837|
| 64|2.4644916643939885|
| 66|2.5766399940083162|
| 67|2.4260711637974204|
| 68| 2.637975941692466|
| 70|2.6068759416924663|
| 71|2.4218581256005125|
| 72|2.2476848370927316|
| 73| 2.226784837092732|
| 74|2.2476848370927316|
| 76| 2.285351503759398|
| 77|2.2476848370927316|
| 78|2.3152618874835857|
| 79| 2.655402412280702|
| 80|  2.45280272556391|
| 82|2.2786070593149543|
| 83|2.4218581256005125|
| 84| 2.410501254975675|
| 86|2.4180532296821817|
| 87|2.4639621896924844|
| 89|1.5790178571428573|
| 91| 2.241784433514889|
| 93| 2.414751582391984|
| 94|  2.42480272556391|
| 95|  2.45280272556391|
| 96|2.2662408521303257|
| 97|2.2449196982841717|
| 98|2.3914245293728724|
| 99| 2.095840852130326|
|100|2.4732351518926583|
|102|2.4298898488131924|
|103| 2.141519698284172|
|104|2.2781756193368037|
|107|2.2584320593149547|
|108|2.3160756193368037|
|109| 2.577900775258216|
|110|2.4435420585824597|
|111|2.4333337252491267|
|112|2.5939898939890984|
|115| 2.234850097162551|
|116|2.6904339203559866|
|118|2.3391990738894095|
|119| 2.678346858232286|
|121| 2.524744201229788|
|124|2.6193961504589724|
|126|2.5424690087689634|
|128|2.4000426392578142|
|129|2.5503720494153717|
|130|2.3557056159118197|
|131|  2.26387079557869|
|132|2.3263606677033977|
|133| 2.491199104760635|
|134|2.5995409746962594|
|135|2.6029163445094663|
+---+------------------+

# Stage 5: Expand on models

After previous optimizations, we have achieved reasonably good result. We wanted to experiment on Linear regression to see if we can achieve better scores

In [ ]:
# Using the Linear Regression on the data
lr = LinearRegression(maxIter=100, regParam=0.3, elasticNetParam=0.8, labelCol='relevance')
pipeline = Pipeline(stages=[lr])

# train the model
model = pipeline.fit(training)

# testing
prediction = model.transform(testing)
prediction.show(5)

In [ ]:
# Writing prediction to a csv file on instance
submission=prediction.select("id","prediction")
submission.show(100)
submission = submission.toPandas()

# output the result into a csv file
submission.to_csv('/home/jiawenz1_c4gcp/answer.csv', index=False)

In [ ]:
+---+------------------+
| id|        prediction|
+---+------------------+
|  1|2.2531105570770817|
|  4| 2.516205260017714|
|  5| 2.412751772787061|
|  6|2.1677275793650796|
|  7| 2.564859307760412|
|  8| 2.212191500385579|
| 10|2.6471629641729106|
| 11| 2.492132776452756|
| 12| 2.482303198701883|
| 13|2.6193777209717926|
| 14| 2.711583790979911|
| 15|  2.57199076228244|
| 19|1.9151565923207223|
| 22|2.6101884256926033|
| 24|2.4518054676506185|
| 25| 2.436263331926154|
| 26| 2.113331976576055|
| 28|2.4895287863373605|
| 29| 2.206066819693464|
| 30|2.3932106557464983|
| 31|2.5763032222239843|
| 32|2.4656891664266207|
| 33| 2.466463631936975|
| 36| 2.424626153453097|
| 39| 2.250550207324413|
| 40|1.7955178571428572|
| 41| 2.473871612864385|
| 42|2.4394108173186435|
| 43| 2.371889957996507|
| 44|2.5031172417230647|
| 45|2.7344864137339733|
| 46|2.6756648451065232|
| 47| 1.764080357142857|
| 49|2.6200956479104724|
| 50|2.5818523734218033|
| 52| 2.556875901291691|
| 53| 2.405162823234474|
| 54| 2.159102978033318|
| 55| 2.606694042533913|
| 56| 2.436363143747295|
| 57|2.4425536634053087|
| 58|  2.48120410904154|
| 59|2.4781460623997367|
| 60|2.0468611119144016|
| 61|2.5714128027112695|
| 62|2.5315011553963203|
| 63| 2.605281762527837|
| 64|2.4644916643939885|
| 66|2.5766399940083162|
| 67|2.4260711637974204|
| 68| 2.637975941692466|
| 70|2.6068759416924663|
| 71|2.4218581256005125|
| 72|2.2476848370927316|
| 73| 2.226784837092732|
| 74|2.2476848370927316|
| 76| 2.285351503759398|
| 77|2.2476848370927316|
| 78|2.3152618874835857|
| 79| 2.655402412280702|
| 80|  2.45280272556391|
| 82|2.2786070593149543|
| 83|2.4218581256005125|
| 84| 2.410501254975675|
| 86|2.4180532296821817|
| 87|2.4639621896924844|
| 89|1.5790178571428573|
| 91| 2.241784433514889|
| 93| 2.414751582391984|
| 94|  2.42480272556391|
| 95|  2.45280272556391|
| 96|2.2662408521303257|
| 97|2.2449196982841717|
| 98|2.3914245293728724|
| 99| 2.095840852130326|
|100|2.4732351518926583|
|102|2.4298898488131924|
|103| 2.141519698284172|
|104|2.2781756193368037|
|107|2.2584320593149547|
|108|2.3160756193368037|
|109| 2.577900775258216|
|110|2.4435420585824597|
|111|2.4333337252491267|
|112|2.5939898939890984|
|115| 2.234850097162551|
|116|2.6904339203559866|
|118|2.3391990738894095|
|119| 2.678346858232286|
|121| 2.524744201229788|
|124|2.6193961504589724|
|126|2.5424690087689634|
|128|2.4000426392578142|
|129|2.5503720494153717|
|130|2.3557056159118197|
|131|  2.26387079557869|
|132|2.3263606677033977|
|133| 2.491199104760635|
|134|2.5995409746962594|
|135|2.6029163445094663|
+---+------------------+